In [1]:
import pandas as pd
import numpy as np

In [2]:
interurbanos = pd.read_excel("../pruebas_out_app/interurbanos.xlsx")
vacs = pd.read_excel("../pruebas_out_app/vacs.xlsx")

In [3]:
print("INTERURBANOS: ",interurbanos.columns, interurbanos.shape)
print("VACs: ",vacs.columns, vacs.shape)

INTERURBANOS:  Index(['Concesion', 'Total_VIA', 'Total_BIT', 'Total_BASE'], dtype='object') (1, 4)
VACs:  Index(['Concesion', 'Total_VAC', 'Total_BIT', 'Total_BASE', 'Total_VIA'], dtype='object') (5, 5)


In [4]:
interurbanos.rename({"Concesion":"CONCESION", "Total_VIA":"VIA","Total_BIT":"BIT","Total_BASE":"BASE"}, axis=1 , inplace=True)
vacs.rename({"Concesion":"CONCESION","Total_VIA":"VIA", "Total_VAC":"VAC","Total_BIT":"BIT","Total_BASE":"BASE"}, axis=1 , inplace=True)

In [5]:
vacs.columns

Index(['CONCESION', 'VAC', 'BIT', 'BASE', 'VIA'], dtype='object')

In [5]:
### INTERURBANOS
# BASE-VIA = BASE-VIA
# %DIF/BASE = ((BASE-VIA)/BASE)*100
# BIT-BASE = BIT-BASE
# %BIT/BASE = (BIT/BASE)*100
# %BIT/VIA = (BIT/VIA)*100

''' 
Los filtros de colores indican:
amarillo si %DIF/BASE => -0.005<val<0.005 # val not in
morado si %BIT/BASE => 0.995<val<1.005 # val in
verde si %BIT/VIA => 0.995<val<1005 # val in
'''
def notificacion():
    print("CE - Contactar Empresa para que aclare las diferencias entre BASE y VIA.")

# def asign_color_interurbanos(df):
#     df["%DIF/BASE"].apply(lambda x: "OK" if -0.005<x<0.005 else x)
#     df["%BIT/VIA"].apply(lambda x: "OK, CE" if 0.995<x<1005 else x) ## ¿Pasamos a comprobar el BIT-BASE?
#     df["BIT-BASE"].apply(lambda x: "OK" if 0.995<x<1.005 else x)

''' 
'CONCESION', 'VIA', 'BIT', 'BASE', 'BASE-VIA', '%DIF/BASE', 'BIT-BASE',
'%BIT/BASE', '%BIT/VIA', 'DECISION'
'''
''' 
amarillo: not -0.005<x["%DIF/BASE"]<0.005
verde: entre 0.995<x["%BIT/VIA"]<1005
morado: entre 0.995<x["%BIT/BASE"]<1.005
'''
def assign_categorie_interurbanos(x): 
    if -0.005<x["%DIF/BASE"]<0.005: ## (UNO BLANCO)
        if 0.995<x["%BIT/VIA"]<1005: ## (UNO BLANCO, A VERDE)
            return pd.Series(["VIA", ""])
        elif not 0.995<x["%BIT/VIA"]<1005: ## (UNO BLANCO, B BLANCO)
            if not -0.005<x["%DIF/BASE"]<0.005: ## DOS AMARILLO
                if 0.995<x["%BIT/VIA"]<1005: ## (DOS AMARILLO, A VERDE)
                    return pd.Series(["VIA", "CE"])
                elif not 0.995<x["%BIT/VIA"]<1005: ## (DOS AMARILLO, B BLANCO)
                    if 0.995<x["%BIT/BASE"]<1.005: ## (DOS AMARILLO, B BLANCO, X MORADO)
                        return pd.Series(["BIT", "ERROR VIA"])
                    else:
                        return pd.Series(["BASE","SIN COINCIDENCIAS"])
    elif not -0.005<x["%DIF/BASE"]<0.005: ## DOS AMARILLO
        if 0.995<x["%BIT/VIA"]<1005: ## (DOS AMARILLO, A VERDE)
            return pd.Series(["VIA", "CE"])
        elif not 0.995<x["%BIT/VIA"]<1005: ## (DOS AMARILLO, B BLANCO)
            if 0.995<x["%BIT/BASE"]<1.005: ## (DOS AMARILLO, B BLANCO, X MORADO)
                return pd.Series(["BIT", "ERROR VIA"])
            else:
                return pd.Series(["BASE","SIN COINCIDENCIAS"])
    else:
        return pd.Series(["CE",""])


### VACs
# BASE-VIA = BASE-VIA
# %DIF/BASE = (BASE-VIA)/BASE
# VAC-BASE = VAC-BASE
# VAC/VIA = VAC/VIA
''' 
amarillo: not -0.005<x["%DIF/BASE"]<0.005
verde: entre 0.995<x["%VAC/VIA"]<1.005
morado: entre x["VAC"]*0.99<x["BIT"]<x["VAC"]*1.01
'''
def assign_categorie_vacs(x):
    if x["VIA"]==0:
        return pd.Series(["VAC",""])
    elif -0.005<x["%DIF/BASE"]<0.005: ## (UNO BLANCO)
        if 0.995<x["%VAC/VIA"]<1.005: ## (UNO BLANCO, A VERDE)
            return pd.Series(["VAC",""])
        elif not 0.995<x["%VAC/VIA"]<1.005: ## (UNO BLANCO, B BLANCO)
            if x["VAC"]*0.99<x["BIT"]<x["VAC"]*1.01: ## (UNO BLANCO, B BLANCO, X MORADO)
                return pd.Series(["ANALIZAR VIA/VAC",""])
            else:
                return pd.Series(["ANALIZAR DATOS",""])
    elif not -0.005<x["%DIF/BASE"]<0.005: ## (DOS AMARILLO)
        if 0.995<x["%VAC/VIA"]<1.005: ## (DOS AMARILLO, A VERDE)
            return pd.Series(["VAC", "CE"])
        elif not 0.995<x["%VAC/VIA"]<1.005: ## (DOS AMARILLO, B BLANCO)
            if x["VAC"]*0.99<x["BIT"]<x["VAC"]*1.01: ## (DOS AMARILLO, B BLANCO, X MORADO)
                return pd.Series(["VAC", "CE"])
            else:
                return pd.Series(["SIN CRITERIO",""])
    else:
        return pd.Series(["CE","CE"])

interurbanos["BASE-VIA"] = interurbanos["BASE"]-interurbanos["VIA"]
interurbanos["%DIF/BASE"] = (interurbanos["BASE-VIA"]/interurbanos["BASE"])*100
interurbanos["BIT-BASE"] = interurbanos["BIT"]-interurbanos["BASE"]
interurbanos["%BIT/BASE"] = (interurbanos["BIT"]/interurbanos["BASE"])*100
interurbanos["%BIT/VIA"] = (interurbanos["BIT"]/interurbanos["VIA"])*100
interurbanos[["DECISION","ACCION"]] = interurbanos.apply(assign_categorie_interurbanos, axis=1)

### VACs
# BASE-VIA = BASE-VIA
# %DIF/BASE = (BASE-VIA)/BASE
# VAC-BASE = VAC-BASE
# VAC/VIA = VAC/VIA

vacs["BASE-VIA"] = vacs["BASE"]-vacs["VIA"]
vacs["%DIF/BASE"] = (vacs["BASE-VIA"]/vacs["BASE"])*100
vacs["VAC-BASE"] = vacs["VAC"]-vacs["BASE"]
vacs["%VAC/VIA"] = (vacs["VAC"]/vacs["VIA"])*100
vacs[["DECISION","ACCION"]] = vacs.apply(assign_categorie_vacs, axis=1)


In [13]:
# print("BASE-VIA", interurbanos["BASE-VIA"].value_counts())
# print("%DIF/BASE", interurbanos["%DIF/BASE"].value_counts())
# print("BIT-BASE", interurbanos["BIT-BASE"].value_counts())
# print("%BIT/BASE", interurbanos["%BIT/BASE"].value_counts())
# print("%BIT/VIA", interurbanos["%BIT/VIA"].value_counts())

In [14]:
# print("BASE-VIA", vacs["BASE-VIA"].value_counts())
# print("%DIF/BASE", vacs["%DIF/BASE"].value_counts())
# print("BIT-BASE", vacs["BIT-BASE"].value_counts())
# print("%BIT/BASE", vacs["%BIT/BASE"].value_counts())
# print("%BIT/VIA", vacs["%BIT/VIA"].value_counts())

In [15]:
interurbanos.to_excel("./pruebas_out_app/interurbanos_table.xlsx")
vacs.to_excel("./pruebas_out_app/vacs_table.xlsx")